In [ ]:
import os

from sqlalchemy import create_engine
import pandas as pd
from multiprocessing.dummy import Pool as ThreadPool

from validator_helper import get_score
from sql import *

def get_engine():
    return create_engine('mysql+pymysql://{user}:{passwd}@{host}/{db}?charset=utf8mb4'.format(
        host='marketingdb-green',
        user=os.environ.get('MARKETINGDB_USER'),
        passwd=os.environ.get('MARKETINGDB_PASSWORD'),
        db='bob_live_mx',
    ))

engine = get_engine()

sql = '''
    select id_catalog_config as id
    from catalog_config
    join items_activity on catalog_config.sku = items_activity.sku
    where visible_in_shop = 1
    limit 1000'''

ids = [element.id for element in engine.execute(sql).fetchall()]

print len(ids)


pool = ThreadPool(16)

def get_score_dict():
    return pool.map(lambda x: get_score(x, engine), ids)

    
%time result = get_score_dict()


pool.close()
pool.join()

print result